In [63]:
from htmd import *
from htmd.builder.builder import *
htmd.config(viewer='vmd')

charmm.listFiles()

In [64]:
from htmd.builder.builder import *
topos  = ['top/top_all36_prot.rtf','top/top_all36_lipid.rtf','top/top_water_ions.rtf','top/top_all36_cgenff.rtf', 'top/cocaine.rtf'] #,'top/ga299.rtf'] #'fluoxetine.rtf']
params = ['par/par_all36_prot.prm','par/par_all36_lipid.prm','par/par_water_ions.prm','par/par_all36_cgenff.prm', 'par/cocaine.prm'] #, 'par/ga299.prm'] #, 'fluoxetine.prm']
streams = ['/Users/waskom/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/str/lipid/toppar_all36_lipid_cholesterol.str']

In [65]:
prot = Molecule('protein.pdb')
prot.filter('protein and noh')

#prot.filter('protein or noh or not resname MOL')
prot.set('segid','P');
#newprot = autoSegment(prot,'protein','P')
#newprot = autoSegment(prot,'protein','P')
prot.center()

2017-02-14 13:03:02,125 - htmd.molecule.molecule - INFO - Removed 4230 atoms. 4249 atoms remaining in the molecule.


In [66]:
lig = Molecule('Cocaine.pdb')
lig.set('segid','L');
lcenter=np.mean(lig.get('coords'),axis=0)
#newlcenter=[np.random.uniform(-10, 10), np.random.uniform(-10, 10),  43 ]
#lig.rotateBy(uniformRandomRotation(), lcenter)
#lig.moveBy(newlcenter-lcenter)
#mol.append(lig)
prot.append(lig)

In [67]:
lig = Molecule('Cholesterol.pdb')
lig.set('segid','C');
lcenter=np.mean(lig.get('coords'),axis=0)
#newlcenter=[np.random.uniform(-10, 10), np.random.uniform(-10, 10),  43 ]
#lig.rotateBy(uniformRandomRotation(), lcenter)
#lig.moveBy(newlcenter-lcenter)
#mol.append(lig)
prot.append(lig)

In [68]:
prot.view()

In [69]:
memb = Molecule('membrane_c36.pdb')
#memb.set('segid','M');
pcenter = np.mean(prot.get('coords','protein'),axis=0)
mcenter = np.mean(memb.get('coords'),axis=0)
memb.moveBy(pcenter-mcenter)
mol = embed(prot,memb)

In [70]:
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='ions', style='VDW')
mol.reps.add(sel='lipids', style='Lines')
mol.view()

In [71]:
coo = mol.get('coords','noh and (lipids or protein)')
m = np.min(coo, axis=0) + [0, 0, -20]
M = np.max(coo, axis=0) + [0, 0, 20]
smol = solvate(mol, minmax=np.vstack((m,M)))
#smol.reps.add(sel='segid L', style='Licorice')
smol.reps.add(sel='water', style='Lines')
smol.view()

2017-02-14 13:03:09,530 - htmd.builder.solvate - INFO - Using water pdb file at: /Users/waskom/miniconda3/lib/python3.5/site-packages/htmd/builder/wat.pdb
2017-02-14 13:03:10,671 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 \


2017-02-14 13:03:48,636 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 27577 water molecules were added to the system.


In [72]:
molbuilt = charmm.build(smol, topo=topos, param=params,stream=streams, 
                        outdir='/users/waskom/Katz_Paper/Simulations/COC_CHO_NEW/build', saltconc=0.15)

2017-02-14 13:04:02,918 - htmd.builder.charmm - WARNING - Multiple (2) rtf topology sections found in /Users/waskom/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/str/lipid/toppar_all36_lipid_cholesterol.str stream file.
2017-02-14 13:04:02,919 - htmd.builder.charmm - WARNING - Multiple (2) prm parameter sections found in /Users/waskom/miniconda3/lib/python3.5/site-packages/htmd/builder/charmmfiles/str/lipid/toppar_all36_lipid_cholesterol.str stream file.
2017-02-14 13:04:14,737 - htmd.builder.charmm - INFO - Writing out segments.
2017-02-14 13:09:13,762 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2017-02-14 13:09:18,965 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2017-02-14 13:09:24,151 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2017-02-14 13:09:29,352 - htmd.m

In [73]:
from htmd.protocols.equilibration_v1 import Equilibration
md = Equilibration()
md.numsteps = 10000000
md.temperature = 310
#md.reference = 'protein and resid 293'
#md.selection = 'segname L and noh'
#md.box = [-25, 25, -25, 25, 43, 45]
#md.k = 5
#md.useconstantratio = True
md.write('/users/waskom/Katz_Paper/Simulations/COC_CHO_NEW/build','DAT_equil1')

2017-02-14 13:18:34,168 - htmd.molecule.readers - WARNING - Reading PDB file with more than 99999 atoms. Bond information can be wrong.
2017-02-14 13:18:34,307 - htmd.molecule.readers - INFO - Non-integer serials were read. For safety we will discard all bond information and serials will be assigned automatically.
2017-02-14 13:18:42,759 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
